In [0]:
%pip install -U langgraph==0.2.57 langchain_community  langchain_experimental databricks-sdk databricks-langchain mlflow databricks-agents>=0.12.0 pydantic 
dbutils.library.restartPython()

In [0]:
# Log the model to MLflow
import os
import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types.llm import CHAT_MODEL_INPUT_SCHEMA, CHAT_MODEL_OUTPUT_SCHEMA

agent_signature = ModelSignature(
    CHAT_MODEL_INPUT_SCHEMA,
    CHAT_MODEL_OUTPUT_SCHEMA,
)

In [0]:
with mlflow.start_run():
    logged_agent_info = mlflow.langchain.log_model(
        lc_model=os.path.join(
            os.getcwd(),
            "02_Multi_Agent_LangGraph_Flow.py",
        ),
        pip_requirements=[
            "langchain",
            "langgraph==0.2.57", 
            "databricks-agents>=0.12.0",
            "pydantic",
            "databricks-langchain",
            "langchain_community",
            "langchain_experimental",
            "databricks-sdk"
        ],
        artifact_path="agent",
        # input_example=input_example,
        signature=agent_signature
    )

In [0]:
# Test the chain locally
logged_agent = mlflow.langchain.load_model(logged_agent_info.model_uri)
# chain.invoke(messages_dict)

In [0]:
input_example = {
        "messages": [
            {
                "role": "user",
                "content": "When did I place the order ORD974?",
            }
        ]
    }
response = logged_agent.invoke(input_example)
print(type(response))

In [0]:
from databricks import agents
import mlflow 

mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "mosaic_agent"
schema = "online_electric_retailer"
model_name = "blog_langgraph"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME)


# Get the Databricks host
host = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Deploy the model to the review app and a model serving endpoint
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version,
              environment_vars={
                "DATABRICKS_TOKEN": token,
                "DATABRICKS_HOST": host },
                tags={"endpointSource": "playground"}
              )
